In [ ]:
import json

with open('./1_word.txt', 'r', encoding='utf-8') as f:
    unigram = json.load(f)


In [ ]:
unigram

In [ ]:
pinyin_word = {pinyin: sum(item['counts']) for pinyin, item in unigram.items()}

In [ ]:
pinyin_word

In [ ]:
with open('./2_word.txt', 'r', encoding='utf-8') as f:
    bigram = json.load(f)

In [ ]:
for i in bigram:
    print(i)
    a = i.strip().split()
    print(a)
    break

In [ ]:

word2pinyin = {}
with open('./word2pinyin.txt', 'r', encoding='utf-8') as f:
    for line in f:
        word, pinyin = line.strip().split()
        word2pinyin[word] = pinyin


In [ ]:
word2pinyin

In [ ]:
def process_input():
    while True:
        try:
            line = input().strip()
            if not line:
                break
            pinyin_list = line.split()
            # 在这里对拼音串进行处理
            # 例如，输出它们或者进行其他操作
            print(pinyin_list)
        except EOFError:
            break

In [ ]:
process_input()

生成1_word.txt文件

In [ ]:
import json
import re
import string
import os
from pypinyin import lazy_pinyin, Style
import time
import sys

start_time1 = time.time()
#得到pinyin->汉字的转化, pinyin2char = {'pinyin': ['汉字1', '汉字2', ...], ...}
with open('../training_data/拼音汉字表.txt', 'r', encoding='gbk') as f:
    pinyin2char = {}
    char2pinyin = {} #方便处理多音字
    for line in f:
        items = line.strip().split()
        pinyin = items[0]
        hanzi = items[1:]
        if pinyin not in pinyin2char:
            pinyin2char[pinyin] = []
        pinyin2char[pinyin].extend(hanzi)
        for word in items[1:]:
            if word not in char2pinyin:
                char2pinyin[word] = []
            char2pinyin[word].append(pinyin)
def is_multi_pinyin(c):
    if c in char2pinyin and len(char2pinyin[c]) > 1:
        return True
    return False


start_time2 = time.time()
#有效的汉字，valid_char = [汉字1, 汉字2, ...]
with open('../training_data/一二级汉字表.txt', 'r', encoding='gbk') as f:
    valid_char = list(f.readline().strip())
end_time2 = time.time()


#处理语料库
start_time3 = time.time()
#得到单字的概率, char_possibility = {'pinyin': {'汉字1': 概率1, '汉字2': 概率2, ...}, ...}
char_possibility = {}

# 定义符号模式
puncs = re.compile(r"\s|[a-zA-Z]|\.|\(|\)|" + "|".join(["，", "。", "、", "：", "；", "？", "！", "（", "）", "《", "》",
                                                       "-", "——", "·", "……", "‘", "’", "“", "”", "/", r"\\", "\\[",
                                                       "\\]", "【", "】", "\\|", "℃"]))

def filter_sentence(sentence):
    # 创建一个包含所有标点符号的字符串
    punctuation = string.punctuation
    # 过滤标点符号和数字
    filtered_sentence = ''.join(char for char in sentence if char not in punctuation and not char.isdigit())
    filtered_sentence = puncs.sub("", filtered_sentence)
    filtered_sentence = re.sub(r'[^\u4e00-\u9fa5]', '', filtered_sentence)
    return filtered_sentence

word_count = {}
def cal_freq(file_name):
    with open(file_name, 'r', encoding='gbk') as f:
        for line in f:
            data = json.loads(line)
            html = data['html']
            title = data['title']
            article = title + html
            article = filter_sentence(article)
            for word in article:
                #if word not in valid_char: continue
                if word not in word_count:
                    word_count[word] = 0
                word_count[word] += 1 
            with open('word_count.json', 'w', encoding = 'utf-8') as f:
                json.dump(word_count, f, ensure_ascii=False, indent=4)
             

file_path = '../语料库/sina_news_gbk'
files = os.listdir(file_path)
for file in files[1:8]:
    print("start")
    cal_freq(os.path.join(file_path, file))
    print("finished this file")

import math
for pinyin in pinyin2char:
    item = {}
    for hanzi in pinyin2char[pinyin]:
        item[hanzi] = word_count.get(hanzi, 1)
    char_possibility[pinyin] = item
with open('char_frequency.json', 'w', encoding='utf-8') as f: 
    f.write(json.dumps(char_possibility, ensure_ascii=False, indent=2))
for pinyin in char_possibility:
    one_pinyin_count = sum(char_possibility[pinyin].values())
    for hanzi in char_possibility[pinyin]:
        char_possibility[pinyin][hanzi] = math.log(one_pinyin_count) - math.log(char_possibility[pinyin][hanzi])

with open('char_possibility.json', 'w', encoding='utf-8') as f:
    json.dump(char_possibility, f, ensure_ascii=False, indent=4)
end_time3 = time.time()
print('处理语料库得到单字概率耗时：', end_time3 - start_time3)
            
            

In [ ]:
import json
import math
with open('../training_data/拼音汉字表.txt', 'r', encoding='gbk') as f:
    pinyin2char = {}
    char2pinyin = {} #方便处理多音字
    for line in f:
        items = line.strip().split()
        pinyin = items[0]
        hanzi = items[1:]
        if pinyin not in pinyin2char:
            pinyin2char[pinyin] = []
        pinyin2char[pinyin].extend(hanzi)
        for word in items[1:]:
            if word not in char2pinyin:
                char2pinyin[word] = []
            char2pinyin[word].append(pinyin)
char_possibility = {}
with open('word_count.json', 'r', encoding='utf-8') as f:
    word_count = json.load(f)
for pinyin in pinyin2char:
    item = {}
    for hanzi in pinyin2char[pinyin]:
        item[hanzi] = word_count.get(hanzi, 1)
    char_possibility[pinyin] = item
with open('char_frequency.json', 'w', encoding='utf-8') as f: 
    f.write(json.dumps(char_possibility, ensure_ascii=False, indent=2))
for pinyin in char_possibility:
    one_pinyin_count = sum(char_possibility[pinyin].values())
    for hanzi in char_possibility[pinyin]:
        char_possibility[pinyin][hanzi] = math.log(one_pinyin_count) - math.log(char_possibility[pinyin][hanzi])

with open('char_possibility.json', 'w', encoding='utf-8') as f:
    json.dump(char_possibility, f, ensure_ascii=False, indent=4)

In [ ]:
pinyin_list = {}
with open('../training_data/拼音汉字表.txt', 'r', encoding='gbk') as f:
    for line in f:
        item = {}
        words = line.strip().split()
        item['words'] = words[1:]
        item['counts'] = []
        for word in words[1:]:
            if word in word_count:
                item['counts'].append(word_count[word])
            else: item['counts'].append(0)
        #item['counts'] = [word_count[word] for word in words[1:]]
        pinyin_list[words[0]] = item

In [ ]:
import json
with open('./1_word.txt', 'w', encoding='utf-8') as f:
    f.write(json.dumps(pinyin_list, ensure_ascii=False, indent=2))

生成2_word.txt文件

In [ ]:
import json

import re
import string
# 定义符号模式
puncs = re.compile(r"\s|[a-zA-Z]|\.|\(|\)|" + "|".join(["，", "。", "、", "：", "；", "？", "！", "（", "）", "《", "》",
                                                       "-", "——", "·", "……", "‘", "’", "“", "”", "/", r"\\", "\\[",
                                                       "\\]", "【", "】", "\\|", "℃"]))

def filter_sentence(sentence):
    # 创建一个包含所有标点符号的字符串
    punctuation = string.punctuation
    # 过滤标点符号和数字
    filtered_sentence = ''.join(char for char in sentence if char not in punctuation and not char.isdigit())
    filtered_sentence = puncs.sub("", filtered_sentence)
    return filtered_sentence

import jieba

def cut(article, two_word_count):
    words = jieba.cut(article)
    for i in range(len(article) - 1):
        word = article[i:i+2]
        if len(word) == 2:
            two_word_count[word] = two_word_count.get(word, 0) + 1

two_word_count = {}
def cal_freq2(file_name):
    with open(file_name, 'r', encoding='gbk') as f:
        for line in f:
            data = json.loads(line)
            html = data['html']
            title = data['title']
            title = filter_sentence(title)
            html = filter_sentence(html)
            cut(title, two_word_count)
            cut(html, two_word_count)
                    
import os
file_path = '../语料库/sina_news_gbk'
files = os.listdir(file_path)
for file in files[1:8]:
    cal_freq2(os.path.join(file_path, file))
            

In [ ]:
from pypinyin import pinyin, Style
result = {}
for item in two_word_count:
    gen = pinyin(item, style=Style.NORMAL)
    if len(gen) < 2: continue
    result[item] = ' '.join(gen[0] + gen[1]) 


In [ ]:
output_dict = {}
    # 统计词频
for word, pin in result.items():
    if pin not in output_dict:
        output_dict[pin] = {'words': [], 'counts': []}
        output_dict[pin]['words'].append(word[0] + ' ' + word[1])
        output_dict[pin]['counts'].append(1)
    else:
        if word in output_dict[pin]['words']:
            output_dict[pin]['counts'][output_dict[pin]['words'].index(word)] += 1
        else:
            output_dict[pin]['words'].append(word[0] + ' ' + word[1])
            output_dict[pin]['counts'].append(1)

In [ ]:
with open('./2_word.txt', 'w', encoding='utf-8') as f:
    f.write(json.dumps(output_dict, ensure_ascii=False, indent=2))

尝试学姐的方式构造

In [ ]:
#得到pinyin->汉字的转化, pinyin2char = {'pinyin': ['汉字1', '汉字2', ...], ...}
with open('./word2pinyin.txt', 'r', encoding='utf-8') as f:
    pinyin2char = {}
    for line in f:
        word, pinyin = line.strip().split()
        if pinyin not in pinyin2char:
            pinyin2char[pinyin] = []
        pinyin2char[pinyin].append(word)

In [ ]:
#得到单字的概率, char_possibility = {'pinyin': {'汉字1': 概率1, '汉字2': 概率2, ...}, ...}
import math
import json
with open('./1_word.txt', 'r', encoding='utf-8') as f:
    unigram = json.load(f)    
one_word_count = {}
for pinyin in unigram:
    if pinyin not in one_word_count:
        one_word_count[pinyin] = 0
    one_word_count[pinyin] += sum(unigram[pinyin]['counts'])
char_possibility = {}
for pinyin in unigram:
    if pinyin not in char_possibility:
        char_possibility[pinyin] = {}
    cp = {}
    for i in range(len(unigram[pinyin]['words'])):
        word = unigram[pinyin]['words'][i]
        count = unigram[pinyin]['counts'][i]
        print("one_word_count: ", one_word_count)
        print("word:", word)
        print("count: ", count)
        poss = math.log(one_word_count[pinyin]) - math.log(count)
        cp[word] = poss
    char_possibility[pinyin] = cp
    print(char_possibility)

In [ ]:
#word_possibility = {'汉字': {'pinyin': {'汉字': 概率, ...}, ...}, ...}    
with open('./2_word.txt', 'r', encoding='utf-8') as f:
    bigram = json.load(f)
two_word_count = {}
for pinyins in bigram:
    if pinyins not in two_word_count:
        two_word_count[pinyins] = 0 
    two_word_count[pinyins] += sum(bigram[pinyins]['counts'])
word_possibility = {}
for pinyin2 in bigram:
    pinyin_list = pinyin2.strip().split()
    word_list = bigram[pinyin2]['words']
    count_list = bigram[pinyin2]['counts']
    for i in range(len(word_list)):
        words = word_list[i]
        word = words.strip().split()
        if word[0] not in word_possibility:
            word_possibility[word[0]] = {}
        if pinyin_list[1] not in word_possibility[word[0]]:
            word_possibility[word[0]][pinyin_list[1]] = {}
        word_possibility[word[0]][pinyin_list[1]][word[1]] = math.log(two_word_count[pinyin2]) - math.log(count_list[i])
    print(word_possibility)